Matrícula: 	564307\
Nível: 	MESTRADO\
Status: 	ATIVO\
E-Mail: 	danieloliveirafff@gmail.com\
Entrada: 	2024.1\
Orientador: 	JOAO PAULO DO VALE MADEIRO\
Área: 	CIÊNCIA DA COMPUTAÇÃO\
Linha de Pesquisa: 	TEORIA DA COMPUTAÇÃO

In [ ]:
# Bibliotecas

# Tratamento de dados
import numpy as np
import pandas as pd

# Gráficos
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette("Accent")
sns.set_style("darkgrid")
import plotly.express as px
import plotly.graph_objects as g

# métricas
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

# Transformação dos dados
# from sklearn.preprocessing import StandardScaler, LabelBinarizer # Normalização dos dados
# from sklearn.model_selection import train_test_split # tratamento dos dados


In [ ]:
# carregando os dados
# motando o google drive

from google.colab import drive
drive.mount('/content/drive')

link = '/content/drive/MyDrive/UFC_mestrado/Sigaa_UFC/1_semestre/aprendizagem_automatica/lista_04_ama/dados/vowel.csv'

df = pd.read_csv(link, header=None)

dados = np.array(df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **TRATAMENTO DE DADOS**

In [ ]:
# separando os dados de x e de y
# X = dados[:, :-1]  # As 8 primeiras colunas são os atributos
# y = dados[:, -1]  # A última coluna é a saída
# y = np.array(y).reshape(-1, 1)

np.random.seed(420) # para manter aleatoriedade

# Separar os atributos (X) e a saída (y)
X = dados[:, :-1]  # As 8 primeiras colunas são os atributos
y = dados[:, -1]   # A última coluna é a saída
y = np.array(y).reshape(-1, 1)



In [ ]:
# Gerar um array de índices embaralhados
indices = np.random.permutation(len(X))

# Usar esses índices para embaralhar X e y de forma consistente
X_embaralhado = X[indices]
y_embaralhado = y[indices]


In [ ]:
# # normalizando os dados com minimo e e max
# from sklearn.preprocessing import MinMaxScaler

# # Normalizando os dados de entrada
# scaler_X = MinMaxScaler()
# X_normalizado = scaler_X.fit_transform(X)

# # Normalizando os dados de saída
# scaler_y = MinMaxScaler()
# y_normalizado = scaler_y.fit_transform(y.reshape(-1, 1))  # y_1 precisa ser 2D para o MinMaxScaler


In [ ]:
# Normalizar X
# scaler = StandardScaler()
# X_normalizado = scaler.fit_transform(X)

# # Binarizar y (supondo que você tenha uma classificação binária)
# binarizer = LabelBinarizer()
# y_binarizado = binarizer.fit_transform(y)


In [ ]:

# Dividir os dados em treino (60%), validação (20%) e teste (20%)
# X_train, X_temp, y_train, y_temp = train_test_split(X_normalizado, y_binarizado, test_size=0.4, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# Normalizando o x com z-score e o y em binarios

normalizacao = lambda x : (x - np.mean(x, axis = 0)) / np.std(x, axis = 0)

# normalizando o x
x_normalizado = normalizacao(X_embaralhado)
x_normalizado.shape

(990, 10)

In [ ]:
# normalizando o y para classes binários

n_classes = np.unique(y_embaralhado)

y_binarios = np.zeros((len(y_embaralhado),len(n_classes))) # quantidade de linhas que tem o y e quantidade de classes

# preenchendo a matrix com 1s nas posições
for i in range(len(y_embaralhado)):
    y_binarios[i, np.where(n_classes == y_embaralhado[i])[0]] = 1

y_binarios.shape

(990, 11)

In [ ]:
# limitando a quantidade de linha para treino, teste e validação do x_normalizado e y_binarios
# Dividir os dados em treino (60%), validação (20%) e teste (20%)

tamanho_treino = int(len(x_normalizado) * 0.6)  # Treino (60%)
tamanho_validacao = int(len(x_normalizado) * 0.2)  # Validação (20%)
tamanho_teste = len(x_normalizado) - tamanho_treino - tamanho_validacao  # Teste (20%)

# Dividir os dados em treino, validação e teste
xtreino = x_normalizado[:tamanho_treino]
ytreino = y_binarios[:tamanho_treino]

xvalidacao = x_normalizado[tamanho_treino:tamanho_treino + tamanho_validacao]
yvalidacao = y_binarios[tamanho_treino:tamanho_treino + tamanho_validacao]

xteste = x_normalizado[tamanho_treino + tamanho_validacao:]
yteste = y_binarios[tamanho_treino + tamanho_validacao:]


# **TREINANDO O MODELO REDE NEURAIS**

In [ ]:
class RedesNeurais:
    def __init__(self):
        self.pesos = []
        self.biases = []
        self.ativacoes = []
        self.ativacoes_deriv = []
        self.momentum_pesos = []  # vetores para o método Adam
        self.momentum_biases = []  # vetorias para o método Adam
        self.v_pesos = []
        self.v_biases = []
        self.t = 0  # Para contar as iterações no Adam
        self.y_train = None  # recebe o y-treino
        self.y_test = None # recebe o y-teste
        self.y_previsto = None  # y calculado
        self.y_val_previsto = None
        self.y_test_previsto = None
        self.loss = []  # Para armazenar as perdas
        self.val_loss = []  # Para armazenar as perdas no conjunto de validação

    def adicionar_camada(self, n_entrada, n_saida, ativacao='relu'):
        # Inicialização dos pesos e bias
        self.pesos.append(np.random.randn(n_entrada, n_saida) * np.sqrt(2. / n_entrada))
        self.biases.append(np.zeros((1, n_saida)))

        # Inicializar momentos e v para Adam
        self.momentum_pesos.append(np.zeros_like(self.pesos[-1]))
        self.momentum_biases.append(np.zeros_like(self.biases[-1]))
        self.v_pesos.append(np.zeros_like(self.pesos[-1]))
        self.v_biases.append(np.zeros_like(self.biases[-1]))

        if ativacao == 'relu':
            self.ativacoes.append(self.relu)
            self.ativacoes_deriv.append(self.relu_derivative)
        elif ativacao == 'linear':
            self.ativacoes.append(self.linear)
            self.ativacoes_deriv.append(self.linear_derivative)
        elif ativacao == 'leaky_relu':
            self.ativacoes.append(self.leaky_relu)
            self.ativacoes_deriv.append(self.leaky_relu_derivative)
        elif ativacao == 'softmax':
            self.ativacoes.append(self.softmax)
            self.ativacoes_deriv.append(None)  # Não é necessário derivada explícita
        else:
            raise ValueError(f"Função de ativação {ativacao} não suportada.")

    def adam_optimizer(self, grad, m, v, beta1, beta2, epsilon, learning_rate):
        m = beta1 * m + (1 - beta1) * grad
        v = beta2 * v + (1 - beta2) * (grad ** 2)
        m_corrigido = m / (1 - beta1 ** (self.t + 1))
        v_corrigido = v / (1 - beta2 ** (self.t + 1))
        return m, v, -learning_rate * m_corrigido / (np.sqrt(v_corrigido) + epsilon)

    def forward(self, X):
        self.entradas = [X]
        self.saidas = []

        for i in range(len(self.pesos)):
            z = np.dot(self.entradas[-1], self.pesos[i]) + self.biases[i]
            a = self.ativacoes[i](z)
            self.entradas.append(z)
            self.saidas.append(a)
        return self.saidas[-1]


    def calcular_perda(self, X, y, learning_rate, regularization_lambda=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        # Cálculo do erro e gradientes
        erro_saida = self.saidas[-1] - y
        grad_saida = erro_saida  # A derivada da perda cross-entropy em relação ao Softmax é o erro

        self.t += 1  # Incrementar o contador de iterações

        # Atualização da camada de saída com Adam e regularização
        self.momentum_pesos[-1], self.v_pesos[-1], delta_pesos = self.adam_optimizer(
            np.dot(self.entradas[-2].T, grad_saida) + regularization_lambda * self.pesos[-1],
            self.momentum_pesos[-1],
            self.v_pesos[-1],
            beta1, beta2, epsilon,
            learning_rate
        )
        self.pesos[-1] += delta_pesos

        self.momentum_biases[-1], self.v_biases[-1], delta_biases = self.adam_optimizer(
            np.sum(grad_saida, axis=0, keepdims=True),
            self.momentum_biases[-1],
            self.v_biases[-1],
            beta1, beta2, epsilon,
            learning_rate
        )
        self.biases[-1] += delta_biases

        erro_propagado = grad_saida

        # Retropropagação nas camadas ocultas com Adam e regularização
        for i in reversed(range(len(self.pesos) - 1)):
            grad_ativacao = self.ativacoes_deriv[i](self.saidas[i])
            erro_propagado = np.dot(erro_propagado, self.pesos[i + 1].T) * grad_ativacao

            self.momentum_pesos[i], self.v_pesos[i], delta_pesos = self.adam_optimizer(
                np.dot(self.entradas[i].T, erro_propagado) + regularization_lambda * self.pesos[i],
                self.momentum_pesos[i],
                self.v_pesos[i],
                beta1, beta2, epsilon,
                learning_rate
            )
            self.pesos[i] += delta_pesos

            self.momentum_biases[i], self.v_biases[i], delta_biases = self.adam_optimizer(
                np.sum(erro_propagado, axis=0, keepdims=True),
                self.momentum_biases[i],
                self.v_biases[i],
                beta1, beta2, epsilon,
                learning_rate
            )
            self.biases[i] += delta_biases

    def treinar(self, X_train, y_train, X_val, y_val, epochs, learning_rate):
        self.loss = []  # Inicializa a lista de perdas
        self.val_loss = []  # Inicializa a lista de perdas de validação
        self.y_train = y_train
        self.y_val = y_val


        for epoch in range(epochs):
            # Forward pass
            self.y_previsto = self.forward(X_train)
            #print(self.y_previsto)
            # Cálculo da perda e retropropagação
            self.calcular_perda(X_train, self.y_train, learning_rate)

            # Cálculo da perda de treinamento
            train_loss = np.mean((self.y_train - self.y_previsto) ** 2)  # MSE

            # Armazenar a perda de treinamento
            self.loss.append(train_loss)

            # Avaliação no conjunto de validação
            self.y_val_previsto = self.forward(X_val)
            val_loss = np.mean((y_val - self.y_val_previsto) ** 2)  # MSE

            # Armazenar a perda de validação
            self.val_loss.append(val_loss)


            # Exibir progresso
            if epoch % 100 == 0:
                print(f'Epoch {epoch}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')


    def prever(self, X):
      return self.forward(X)

    #________________________Métricas____________________
    def metricas(self,para,tipo):
        if self.y_train is None or self.y_previsto is None:
            raise ValueError("Você precisa treinar a rede antes de calcular as métricas.")



        #y_pred = np.argmax(self.y_previsto, axis=1)
        #y_true = np.argmax(y_train, axis=1)
        def retorna_metricas(y_true, y_pred, tipo):
            acuracia = accuracy_score(y_true, y_pred)
            recall = recall_score(y_true, y_pred, average='weighted')
            f1 = f1_score(y_true, y_pred, average='weighted')
            precisao = precision_score(y_true, y_pred, average='weighted')
            if tipo == 'acuracia':
                return acuracia
            elif tipo == 'recall':
                return recall
            elif tipo == 'f1':
                return f1
            elif tipo == 'precisao':
                return precisao
            elif tipo == "classificacao":
                tabela = pd.DataFrame({
                'Acuracia': [acuracia],
                'Precisao': [precisao],
                'Recall': [recall],
                'F1_score': [f1]
                })
                tabela_estilo = tabela.style.format('{:.2f}')
                tabela_estilo.set_table_styles([
                    {'selector': 'th', 'props': 'font-family: Helvetica; color: #dddd55; background-color: #34495E; text-align: center;'},
                    {'selector': 'td', 'props': 'font-family: Helvetica; color: white; background-color: #34495E; text-align: left;'}
                ])
                return tabela_estilo
            else:
              raise ValueError(f"Métrica {tipo} não suportada.")
            # return self.retorna_metricas(y_true, y_pred, tipo)

        #y_pred = np.argmax(self.saidas[-1], axis=1)
        # y_true = np.argmax(self.y_train, axis=1)
        #y_true = self.y_train.flatten()
        #y_pred = np.argmax(self.y_previsto, axis=1)
        # y_true = self.y_train
        # y_pred = self.y_previsto
        # Converter y_true e y_pred de one-hot para rótulos

        if para == 'treino':
          y_true = np.argmax(self.y_train, axis=1)
          y_pred = np.argmax(self.y_previsto, axis=1)
          return retorna_metricas(y_true,y_pred,tipo)
        elif para == 'validacao':
          y_true = np.argmax(self.y_val, axis=1)
          y_pred = np.argmax(self.y_val_previsto, axis=1)
          return retorna_metricas(y_true,y_pred, tipo)
        elif para == 'teste':
          y_true = np.argmax(self.y_test, axis=1)
          y_pred = np.argmax(self.y_test_previsto, axis=1)
          return retorna_metricas(y_true,y_pred, tipo)


    def calcular_metricas(self, y_true, y_predito):
        acuracia = accuracy_score(y_true, y_predito)
        recall = recall_score(y_true, y_predito, average='macro')  # Ou outro tipo de média, conforme necessário
        precisao = precision_score(y_true, y_predito, average='macro')  # Ou outro tipo de média, conforme necessário
        return acuracia, recall, precisao



    #________________________Funções de calculo____________________
    @staticmethod
    def linear(x):
        return x

    @staticmethod
    def relu(x):
        return np.maximum(0, x)

    @staticmethod
    def leaky_relu(x, alpha=0.01):
        return np.where(x > 0, x, alpha * x)

    @staticmethod
    def linear_derivative(x):
        return np.ones_like(x)

    @staticmethod
    def relu_derivative(x):
        return np.where(x > 0, 1, 0)

    @staticmethod
    def leaky_relu_derivative(x, alpha=0.01):
        return np.where(x > 0, 1, alpha)

    @staticmethod
    def softmax(x):
        # Subtrai o valor máximo de cada linha para melhorar a estabilidade numérica
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        # Calcula a soma das exponenciais para cada linha
        resultado = exp_x / np.sum(exp_x, axis=1, keepdims=True)
        return resultado

    def get_softmax(self, camada):

        if camada < 0 or camada >= len(self.saidas):
            raise IndexError("Índice da camada fora do intervalo.")
        return self.softmax(self.saidas[camada])


    #________________________gráficos________________________________
    def grafico_perda(self):
          if not hasattr(self, 'loss') or self.loss is None:
              raise ValueError("A lista de perdas 'self.loss' não está definida.")

          if isinstance(self.loss, np.ndarray):
              self.loss = self.loss.tolist()  # Converte numpy array para lista, se necessário

          if isinstance(self.val_loss, np.ndarray):
              self.val_loss = self.val_loss.tolist()  # Converte numpy array para lista, se necessário


          # Criar uma figura vazia
          fig = px.line(title='Gráfico Loss', labels={'x': 'Épocas', 'y': 'Loss'}, line_shape='linear')

          # Adicionar a linha de treino com a legenda "Treino"
          fig.add_scatter(x=list(range(len(self.loss))), y=self.loss, mode='lines', name='Treino', line=dict(width=6))

          # Adicionar a linha de validação com a legenda "Validação"
          fig.add_scatter(x=list(range(len(self.val_loss))), y=self.val_loss, mode='lines', name='Validação', line=dict(width=4))


          fig.update_layout(
              xaxis_title='Épocas',
              yaxis_title='Loss',
              hovermode='x',
              template='plotly_white',
              width=1400,
              height=600,
              margin=dict(l=20, r=20, t=35, b=20),
              title_font=dict(size=20, color='black'),
              xaxis=dict(title=dict(font=dict(size=14, color='black')), tickfont=dict(color='black')),
              yaxis=dict(title=dict(font=dict(size=14, color='black')), tickfont=dict(color='black'))
          )

          return fig.show()


    def grafico_comparacao(self):
        y_true = np.argmax(self.y_train, axis=1)
        y_pred = np.argmax(self.y_previsto, axis=1)

        # if not hasattr(self, 'y') or y_true is None or not hasattr(self, 'y_previsto') or y_pred is None:
        #     raise ValueError("Os dados para comparação 'y_true' ou 'y_pred' não estão definidos.")

        # if isinstance(y_true, np.ndarray):
        # y_true = y_true.tolist()  # Converte numpy array para lista, se necessário

        # if isinstance(y_pred, np.ndarray):
        # y_pred = y_pred.tolist()  # Converte numpy array para lista, se necessário

        # Criar uma lista de índices com o mesmo tamanho de 'y'
        indices = list(range(len(y_true)))

        # Criação do DataFrame com os dados
        data = {
            'Índices': indices,
            'Y': y_true.flatten(),
            'PREVISTO': y_pred.flatten()
        }

        df = pd.DataFrame(data)

        # Criar o gráfico utilizando o DataFrame
        fig = px.line(df, x='Índices', y=['Y', 'PREVISTO'], title='Comparação de y com o y previsto')

        fig.update_layout(
            xaxis_title='Índices',
            yaxis_title='Valores',
            hovermode='x',
            template='plotly_white',
            width=1500,
            height=400,
            margin=dict(l=20, r=20, t=35, b=20),
            title_font=dict(size=20, color='black'),
            xaxis=dict(title=dict(font=dict(size=14, color='black')), tickfont=dict(color='black')),
            yaxis=dict(title=dict(font=dict(size=14, color='black')), tickfont=dict(color='black'))
        )

        return fig.show()

    def histograma_frequencia(self):
          y_true = np.argmax(self.y_train, axis=1)
          y_pred = np.argmax(self.y_previsto, axis=1)
          data = {
              'Valores':np.concatenate((y_true, y_pred)),
              'Tipo': ['Y'] * len(y_true) + ['PREVISTO'] * len(y_pred)
              }

          df = pd.DataFrame(data)

          # Criar um histograma para visualizar a frequência dos valores
          fig = px.histogram(df, x='Valores', color='Tipo', barmode='overlay', title='Histograma de Frequências de y_treino e y_previsto')

          fig.update_layout(
              xaxis_title='Valores',
              yaxis_title='Frequência',
              template='plotly_white',
              width=1200,
              height=400,
              title_font=dict(size=20, color='black'),
              xaxis=dict(title=dict(font=dict(size=14, color='black')), tickfont=dict(color='black')),
              yaxis=dict(title=dict(font=dict(size=14, color='black')), tickfont=dict(color='black'))
          )
          return fig.show()


In [ ]:
# rede_neural = RedesNeurais()

# n_features = 10  # Número de características de entrada
# n_classes = len(np.unique(y))  # Número de classes

# # Adicionando camadas: entrada (10 neurônios), ocultas (8 neurônios cada) e saída (n_classes neurônios)
# rede_neural.adicionar_camada(n_features, 4, ativacao='linear')  # Primeira camada oculta
# rede_neural.adicionar_camada(4, 4, ativacao='linear')  # Segunda camada oculta
# rede_neural.adicionar_camada(4, n_classes, ativacao='softmax')  # Camada de saída para classificação multiclasse

# # Treinando a rede
# rede_neural.treinar(X_train, y_train, X_val, y_val, epochs=5000, learning_rate=0.001)


In [ ]:
rede_neural = RedesNeurais()

n_features = 10  # Número de características de entrada
n_classes = len(np.unique(y))  # Número de classes

# Adicionando camadas: entrada (10 neurônios), ocultas (8 neurônios cada) e saída (n_classes neurônios)
rede_neural.adicionar_camada(n_features, 4, ativacao='linear')  # Primeira camada oculta
rede_neural.adicionar_camada(4, 4, ativacao='linear')  # Segunda camada oculta
rede_neural.adicionar_camada(4, n_classes, ativacao='softmax')  # Camada de saída para classificação multiclasse

rede_neural.treinar(xtreino, ytreino, xvalidacao, yvalidacao, epochs=5000, learning_rate=0.001)

Epoch 0, Train Loss: 0.1176, Validation Loss: 0.1169
Epoch 100, Train Loss: 0.0994, Validation Loss: 0.1002
Epoch 200, Train Loss: 0.0870, Validation Loss: 0.0882
Epoch 300, Train Loss: 0.0799, Validation Loss: 0.0812
Epoch 400, Train Loss: 0.0756, Validation Loss: 0.0768
Epoch 500, Train Loss: 0.0726, Validation Loss: 0.0737
Epoch 600, Train Loss: 0.0701, Validation Loss: 0.0710
Epoch 700, Train Loss: 0.0676, Validation Loss: 0.0681
Epoch 800, Train Loss: 0.0649, Validation Loss: 0.0651
Epoch 900, Train Loss: 0.0621, Validation Loss: 0.0621
Epoch 1000, Train Loss: 0.0591, Validation Loss: 0.0592
Epoch 1100, Train Loss: 0.0561, Validation Loss: 0.0566
Epoch 1200, Train Loss: 0.0534, Validation Loss: 0.0543
Epoch 1300, Train Loss: 0.0513, Validation Loss: 0.0526
Epoch 1400, Train Loss: 0.0497, Validation Loss: 0.0513
Epoch 1500, Train Loss: 0.0484, Validation Loss: 0.0504
Epoch 1600, Train Loss: 0.0475, Validation Loss: 0.0497
Epoch 1700, Train Loss: 0.0467, Validation Loss: 0.0491
Epoc

In [ ]:
rede_neural.grafico_perda()

In [ ]:
rede_neural.histograma_frequencia()

In [ ]:
# Fazer previsões
y_train_previsto = rede_neural.prever(xtreino)
y_val_previsto = rede_neural.prever(xvalidacao)
y_test_previsto = rede_neural.prever(xteste)
rede_neural.y_test = yteste
# Armazenar as previsões dentro da classe
rede_neural.y_previsto = y_train_previsto
rede_neural.y_val_previsto = y_val_previsto
rede_neural.y_test_previsto = y_test_previsto

# Calcular e imprimir métricas para o conjunto de treino
print("Treinamento:")
print("Acurácia:", rede_neural.metricas('treino', 'acuracia'))
print("Recall:", rede_neural.metricas('treino', 'recall'))
print("Precisão:", rede_neural.metricas('treino', 'precisao'))

# Calcular e imprimir métricas para o conjunto de validação
print("\nValidação:")
print("Acurácia:", rede_neural.metricas('validacao', 'acuracia'))
print("Recall:", rede_neural.metricas('validacao', 'recall'))
print("Precisão:", rede_neural.metricas('validacao', 'precisao'))

# Calcular e imprimir métricas para o conjunto de teste
print("\nTeste:")
print("Acurácia:", rede_neural.metricas('teste', 'acuracia'))
print("Recall:", rede_neural.metricas('teste', 'recall'))
print("Precisão:", rede_neural.metricas('teste', 'precisao'))


Treinamento:
Acurácia: 0.7171717171717171
Recall: 0.7171717171717171
Precisão: 0.7182875379782415

Validação:
Acurácia: 0.702020202020202
Recall: 0.702020202020202
Precisão: 0.7173468299696594

Teste:
Acurácia: 0.7222222222222222
Recall: 0.7222222222222222
Precisão: 0.7342802043470492


In [ ]:


# Cálculo das métricas para o conjunto de treino
rede_neural.metricas('treino','classificacao')

,Acuracia,Precisao,Recall,F1_score
0,0.72,0.72,0.72,0.72


In [ ]:
# Cálculo das métricas para o conjunto de validação
rede_neural.metricas('validacao','classificacao')

,Acuracia,Precisao,Recall,F1_score
0,0.70,0.72,0.70,0.70


In [ ]:
# Cálculo das métricas para o conjunto de teste
rede_neural.metricas('teste','classificacao')

,Acuracia,Precisao,Recall,F1_score
0,0.72,0.73,0.72,0.72
